In [ ]:
import os
import nmslib

# towhee is a library that converts images into vectors
from towhee import AutoPipes

# HNSW initialization with Euclidean operations
index = nmslib.init(method = 'hnsw', space = 'l2')
p = AutoPipes.pipeline('image-embedding')

path = './JPEGImages'
# get a list of all the images in the JPEGImages directory
dir_list = os.listdir(path)

length = len(dir_list)

for i in range(0, length):
    print(f'{i}/{length}')
    
    # JPEGImages/currentFile
    filePath = f'{path}/{dir_list[i]}'
    res = p(filePath)
    
    # adds a node into HNSW
    index.addDataPoint(i, res.get())
index.createIndex({ 'post': 2 })

# True means image and database is saved
index.saveIndex('imageDB', True)

# Test that one of the images was uploaded to the database
query = p('./JPEGImages/2012_004330.jpg').get()

k = 5

ids, distances = index.knnQuery(query, k=k)

# Print the results
print("Query Point:", query)
print("Nearest Neighbors:")
for i, idx in enumerate(ids):
    print("Index:", idx, "Distance:", distances[i])

In [ ]:
import os
import nmslib
from towhee import AutoPipes
import time

# PIL (pillow) allows rendering of images in Python
from PIL import Image

index = nmslib.init(method = 'hnsw', space = 'l2')
p = AutoPipes.pipeline('image-embedding')

path = './JPEGImages'
dir_list = os.listdir(path)

index.loadIndex('imageDB', True)

# vectorize a form of an image through its URL to be searched
# URL can be changed to the photo you would like to search for
query = p('https://cf.ltkcdn.net/aquariums/fish-names/images/orig/324592-1600x1066-goldfish-swimming-in-fish-tank.jpg').get()

# find 5 nearest neighbors
k = 5

ids, distances = index.knnQuery(query, k = k)

print("Query Point:", query)
print("Nearest Neighbors:")
for i, idx in enumerate(ids):
    print("Index:", idx, "Distance:", distances[i])
    image = Image.open(f'{path}/{dir_list[idx]}')
    image.show()
    time.sleep(1)